# High-throughput first-principles calculations
This notebook outlines the procedure for calculating the thermodynamic stabiliy of the structures using DFT, and the bandgap using hybrid-dft. It's implementation requires: 
- [Atomate](http://atomate.org) and [Fireworks](https://materialsproject.github.io/fireworks/)
- A [MongoDB](https://www.mongodb.com) database setup to use as a Fireworks Launchpad
- Access to the [Vienna Ab-initio Simulation Package (VASP)](https://www.vasp.at/)


**You can [skip forward](#analysis) where the final vasp output files are read in to get bandgaps of final candidate materials.**

In [1]:
### Imports ###
import json
import os
from tqdm import tqdm
from itertools import zip_longest

from pymatgen import Structure, MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDEntry
from pymatgen.analysis.reaction_calculator import ComputedReaction
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.io.vasp.sets import DictSet
from matgendb import QueryEngine

from fireworks import LaunchPad
from atomate.vasp.workflows.presets.core import wf_structure_optimization
from atomate.vasp.powerups import add_modify_incar, add_additional_fields_to_taskdocs
from pymatgen.io.vasp.outputs import Vasprun

API_key = os.environ.get('MP_API_KEY')
m = MPRester(API_key)

## Thermodynamic stability
We determine energy above the convex hull via the following procedure:
1. Import the new compounds and submit them for structural relaxation calculations. 
2. Determine their competing phases and submit those for relaxation under the same calculation conditions. 
3. Use total energies of all relaxed structures to compute energy above the convex hull
4. VASP input sets are created to calculate the bandgap of those that are within 0.1 eV of the convex hull.


### Candidate compounds

In [2]:
# Import structures
with open('data/structures_published.json', 'r') as f:
    quaternary_oxides_to_calc = json.load(f)

# Convert back to pymatgen Structure objects
quaternary_oxides_to_calc = [{'structure': Structure.from_dict(i)} 
                             for i in quaternary_oxides_to_calc]

In [ ]:
# Set up fireworks launchpad 
launchpad = LaunchPad(host="YOUR_HOSTNAME", port=27017, 
                      username="username", password="password", 
                      name="databasename")

# Add unique indentifiers
for n, i in enumerate(quaternary_oxides_to_calc):
    i['solaroxide_id'] = 'solaroxide_candidate_{0}'.format(n)

# Add each structure to the launchpad as a modified workflow 
for i in quaternary_oxides_to_calc:
    struct = i['structure']
    orig_wf = wf_structure_optimization(struct)
    modified_wf = add_modify_incar(orig_wf, modify_incar_params={'incar_update': {'EDIFFG': -0.1, 'ALGO': 'ALL',
                                                            'EDIFF': 0.000001, 'ENCUT': 600,
                                                            'KPAR': 3, 'NCORE': 24}})
    modified_wf = add_additional_fields_to_taskdocs(modified_wf, 
                                                update_dict={'solaroxide_id': i['solaroxide_id']})

    launchpad.add_wf(modified_wf)

### Competing phases
We use all compounds in the materials project associated with an ICSD entry as a pool of possible competing phases. We download these directly, which can take some time.

In [ ]:
def grouper(iterable, n, fillvalue=None):
    #Collect data into fixed-length chunks or blocks
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

# Submit MP query
ICSD_ids = m.query(criteria={'icsd_ids.0': {'$exists': True}}, properties=['task_id'])
ICSD_ids = [e['task_id'] for e in ICSD_ids]
MP_competing_phases = []
mpid_groups = [g for g in grouper(ICSD_ids, 1000)]

# Download structures and associated energies in chunks
for group in tqdm(mpid_groups):
    mpid_list = list(filter(None, group))
    entries = m.query({"task_id": {"$in": mpid_list}}, ['task_id','final_energy_per_atom',
                                                        'structure'])
    MP_competing_phases.extend(entries)

# Rename some dict keys
all_competing_phases = [{'structure': i['structure'], 'id': i['task_id'],
                        'energy_atom': i['final_energy_per_atom']} for i in MP_competing_phases]

In [ ]:
# Add chemsys and tot_energy keys to both list of candidates and competing phases
for i in quaternary_oxides_to_calc:
    i['chemsys'] = i['structure'].composition.element_composition.elements
    # Dummy, large energy as we don't know it yet
    i['tot_energy'] = 50.

for struc in all_competing_phases:
    struc['chemsys'] = set(struc['structure'].composition.element_composition.elements)
    struc['tot_energy'] = struc['structure'].composition.num_atoms * struc['energy_atom']

In [ ]:
# Create phase diagram objects to get potential decomposition products
for candidate in quaternary_oxides_to_calc:
    # Get competing phases for this system
    competition = [i for i in all_competing_phases if i['chemsys'].issubset(candidate['chemsys'])]
    entries = [PDEntry(i['structure'].composition, i['tot_energy'],
                      name=i['id']) for i in competition]
    
    # Add the new system in
    new_entry = PDEntry(candidate['structure'].composition.element_composition, 
                          candidate['tot_energy'], attribute = 'new') 
    entries.append(new_entry)
    
    # Make phase diagram and get decomposition products
    pd = PhaseDiagram(entries)
    decomp_prods = pd.get_decomp_and_e_above_hull(new_entry)
    decomp_ids = [i.name for i in decomp_prods[0].keys()]
    candidate['decomp_prods'] = [{ 'structure': i['structure'], 
                                 'tot_energy': i['tot_energy'],
                                 'id': i['id']} for i in competition if i['id'] in decomp_ids]

In [ ]:
# Work out what set of decomposition products we need to calculate energies for
decomp_prod_ids = []
for i in quaternary_oxides_to_calc:
    print(i['structure'].composition.reduced_formula)
    for prod in i['decomp_prods']:
        print("{0:10}   {1:10}   {2}".format(prod['structure'].composition.reduced_formula, 
                                             prod['id'], prod['tot_energy']))
        decomp_prod_ids.append(prod['id'])
decomp_prod_ids = set(decomp_prod_ids)

# Submit calcs in the same way as for candidate compounds
decomp_prod_set = []
for n,i in enumerate(decomp_prod_ids):
    for j in all_competing_phases:
        if i == j['id']:
            struc = j['structure']
    decomp_prod_set.append({'solaroxide_id': 'Solaroxide_decompprod_{}'.format(n),
                            'MP_id': i, 'structure': struc })
    
for i in decomp_prod_set:
    struc = i['structure']
    orig_wf = wf_structure_optimization(struc)
    modified_wf = add_modify_incar(orig_wf, modify_incar_params={'incar_update': {'EDIFFG': -0.1, 'ALGO': 'ALL',
                                                            'EDIFF': 0.000001, 'ENCUT': 600,
                                                            'KPAR': 3, 'NCORE': 24}})
    modified_wf = add_additional_fields_to_taskdocs(modified_wf, 
                                                   update_dict={'DD_struc_id': i['DD_struc_id']})
    launchpad.add_wf(modified_wf)

## Analysis
The total energies of candidates and competing phases can now be used to work out values of energy above the convex hull.

In [ ]:
# Set up query engine to get energies from database 
qe = QueryEngine(host="YOUR_HOSTNAME" port=27017, database="databasename", collection="tasks", 
                 user="username", password="password")

# Get candidate structure info
candidate_strucs = qe.query(criteria= {'solaroxide_id': {'$regex': 'solaroxide_candidate'}}, 
                  properties=["formula_pretty", "task_id", "output.structure", 
                              "solaroxide_id", "output.energy", "output.energy_per_atom"])
candidate_strucs = list(candidate_strucs)

# And of decomp products
decomp_prod_set_DFT = qe.query(criteria= {'solaroxide_id': {'$regex': 'Solaroxide_decompprod_'}}, 
                  properties=["formula_pretty", "task_id", "output.structure", 
                              "DD_struc_id", "output.energy", "output.energy_per_atom"])
decomp_prod_set_DFT = list(decomp_prod_set_DFT)

# Insert calculated total energy values into original list 
for i in quaternary_oxides_to_calc:
    for e in candidate_strucs:
        if e['solaroxide_id'] == i['solaroxide_id']:
            i['tot_energy'] = e['output.energy']

# And for decomp prods
for i in quaternary_oxides_to_calc:
    for prod in i['decomp_prods']:
        form = prod['structure'].composition.reduced_formula
        for e in decomp_prod_set_DFT:
            if e['formula_pretty'] == form:
                prod['tot_energy'] = e['output.energy']

In [ ]:
# Calculate ehull values
stable_count = 0
metastable_count = 0
unstable_count = 0
ehull_values = []
for struc in quaternary_oxides_to_calc:
    comp = struc['structure'].formula
    print(comp)
    candidate_entry = [ComputedEntry(comp, struc['tot_energy'])]
    spacegroup = struc['structure'].get_space_group_info()
    
    decomp_entries = []
    for decomp_prod in struc['decomp_prods']:
        comp_decomp = decomp_prod['structure'].composition
        print(comp_decomp)
        entry = ComputedEntry(comp_decomp, decomp_prod['tot_energy'])
        decomp_entries.append(entry)
        
    # Use the pymatgen reaction tool 
    # This energy is calculated from the reduced formulas as written in the reaction
    reaction = ComputedReaction(candidate_entry, decomp_entries)
    ehull = -reaction.calculated_reaction_energy/reaction.all_comp[0].num_atoms
    struc['ehull_DFT'] = ehull
    struc['spacegroup'] = spacegroup[0]
    
    print("Reaction deltaE {0:7.2f}  |   {1}  | e_hull/atom  {2:.3f}".format(reaction.calculated_reaction_energy, reaction,
                                                                        ehull))
    print('------------------------- -----------------------------------')
    print("Candidate: {0:8}  num_atoms (POSCAR): {1}  Energy (OUTCAR): {2:.2f}".format(candidate_entry[0].composition.reduced_formula,
                                                                                           int(candidate_entry[0].composition.num_atoms),
                                                                                          candidate_entry[0].energy))
    for decomp_entry in decomp_entries:
        print("Product:   {0:8}  num_atoms (POSCAR): {1:2}  Energy (OUTCAR): {2:.2f}".format(decomp_entry.composition.reduced_formula,
                                                                                           int(decomp_entry.composition.num_atoms),
                                                                                          decomp_entry.energy))
    print("=============================================================")
    print(' ')
    if ehull < 0:
        stable_count += 1 
    elif ehull > 0.1:
        unstable_count += 1 
    else:
        metastable_count += 1
        
print ('*******************************')
print("Total Stable:     {0:3}    e_hull = 0 eV".format(stable_count))
print("Total metastable: {0:3}    0 < e_hull < 0.l eV".format(metastable_count))
print("Total unstable:   {0:3}    e_hull > 0.1 eV".format(unstable_count))
print ('*******************************')

## Bandgap calculation
Finally, we generate vasp input files of the metastable and stable compositions to calculate the bandgap.

In [ ]:
meta_and_stable = [i for i in quaternary_oxides_to_calc if i['ehull_DFT'] <= 0.1]

# Insert relaxed structure into metastable results
for i in meta_and_stable:
    for e in candidate_strucs:
        if e['solaroxide_id'] == i['solaroxide_id']:
            i['relaxed_structure'] = Structure.from_dict(e['output.structure'])

# Define the calculation parameters 
calc_config = {'INCAR': {'ALGO': 'ALL', 'GGA': 'PE', 'EDIFF': 1e-06, 'EDIFFG': -0.01, 'ENMAX': 550,
                  'ISMEAR': 0, 'ISPIN': 2, 'LORBIT': 11, 'LREAL': 'AUTO', 'NELM': 100, 
                  'NSW': 0, 'PREC': 'Accurate', 'SIGMA': 0.1, 'NCORE': 24, 'KPAR': 8,
                  'LHFCALC': '.TRUE.', 'PRECFOCK': 'fast', 'AEXX': 0.25, 
                  'TIME': 0.30, 'HFSCREEN': 0.207,
                 'MAGMOM': {'Ce': 5, 'Ce3+': 1, 'Co': 5, 'Co3+': 0.6, 'Co4+': 1, 'Cr': 5, 
                            'Dy3+': 5, 'Er3+': 3, 'Eu': 10, 'Eu2+': 7, 'Eu3+': 6, 'Fe': 5, 
                            'Gd3+': 7, 'Ho3+': 4, 'La3+': 0.6, 'Lu3+': 0.6, 'Mn': 5, 'Mn3+': 4, 
                            'Mn4+': 3, 'Mo': 5, 'Nd3+': 3, 'Ni': 5, 'Pm3+': 4, 'Pr3+': 2, 'Sm3+': 5, 
                            'Tb3+': 6, 'Tm3+': 2, 'V': 5, 'W': 5, 'Yb3+': 1}},
        'KPOINTS': {'reciprocal_density': 64}, 
        'POTCAR': {'Ac': 'Ac', 'Ag': 'Ag', 'Al': 'Al', 'Ar': 'Ar', 'As': 'As', 'Au': 'Au', 'B': 'B', 
                   'Ba': 'Ba_sv', 'Be': 'Be_sv', 'Bi': 'Bi', 'Br': 'Br', 'C': 'C', 'Ca': 'Ca_sv', 
                   'Cd': 'Cd', 'Ce': 'Ce', 'Cl': 'Cl', 'Co': 'Co', 'Cr': 'Cr_pv', 'Cs': 'Cs_sv', 
                   'Cu': 'Cu_pv', 'Dy': 'Dy_3', 'Er': 'Er_3', 'Eu': 'Eu', 'F': 'F', 'Fe': 'Fe_pv', 
                   'Ga': 'Ga_d', 'Gd': 'Gd', 'Ge': 'Ge_d', 'H': 'H', 'He': 'He', 'Hf': 'Hf_pv', 
                   'Hg': 'Hg', 'Ho': 'Ho_3', 'I': 'I', 'In': 'In_d', 'Ir': 'Ir', 'K': 'K_sv', 
                   'Kr': 'Kr', 'La': 'La', 'Li': 'Li_sv', 'Lu': 'Lu_3', 'Mg': 'Mg_pv', 'Mn': 
                   'Mn_pv', 'Mo': 'Mo_pv', 'N': 'N', 'Na': 'Na_pv', 'Nb': 'Nb_pv', 'Nd': 'Nd_3', 
                   'Ne': 'Ne', 'Ni': 'Ni_pv', 'Np': 'Np', 'O': 'O', 'Os': 'Os_pv', 'P': 'P', 
                   'Pa': 'Pa', 'Pb': 'Pb_d', 'Pd': 'Pd', 'Pm': 'Pm_3', 'Pr': 'Pr_3', 'Pt': 'Pt', 
                   'Pu': 'Pu', 'Rb': 'Rb_sv', 'Re': 'Re_pv', 'Rh': 'Rh_pv', 'Ru': 'Ru_pv', 'S': 'S', 
                   'Sb': 'Sb', 'Sc': 'Sc_sv', 'Se': 'Se', 'Si': 'Si', 'Sm': 'Sm_3', 'Sn': 'Sn_d', 
                   'Sr': 'Sr_sv', 'Ta': 'Ta_pv', 'Tb': 'Tb_3', 'Tc': 'Tc_pv', 'Te': 'Te', 'Th': 'Th', 
                   'Ti': 'Ti_pv', 'Tl': 'Tl_d', 'Tm': 'Tm_3', 'U': 'U', 'V': 'V_pv', 'W': 'W_pv', 
                   'Xe': 'Xe', 'Y': 'Y_sv', 'Yb': 'Yb_2', 'Zn': 'Zn', 'Zr': 'Zr_sv'}}

# Distribute the necessary files
for n,i in enumerate(meta_and_stable):
    dirname = './BG_CALCS/{0}'.format(i['solaroxide_id'])
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    v  = DictSet(i['relaxed_structure'], config_dict = calc_config)
    v.write_input(dirname)

The generated sets of vasp input files are run, then outputs are parsed below.

<a id='analysis'></a>
## Bandgap values

In [3]:
# Bandgap analysis
directory = 'data/BG_CALCS'
print(directory,".....")
for root, subdirs, files in os.walk(directory):
    if subdirs:
        for subdir in subdirs:
            try:
                vasprun = Vasprun(os.path.join(directory,subdir,'vasprun.xml'))
                print(subdir, vasprun.as_dict()['pretty_formula'], 
                      vasprun.converged_electronic, vasprun.converged_ionic, 
                      vasprun.eigenvalue_band_properties[0])
            except:
                print('XXXX NOPE for {0} {1}'.format(subdir,
                                                   vasprun.as_dict()['pretty_formula']))

data/BG_CALCS .....
Solaroxides_June18_0 MgFe(SO4)2 True True 4.065099999999999
Solaroxides_June18_125 Li2MnSiO5 True True 2.2393
Solaroxides_June18_188 MnCd(GeO3)2 True True 2.523
Solaroxides_June18_189 MnCd(GeO3)2 True True 2.4728999999999997
Solaroxides_June18_190 MnCd(GeO3)2 True True 1.7599999999999998
Solaroxides_June18_197 ZrMnSi2O7 True True 4.367900000000001
Solaroxides_June18_203 ZrMnSi2O7 True True 4.320499999999999
Solaroxides_June18_204 ZrMnSi2O7 True True 3.9518999999999993
Solaroxides_June18_205 ZrMnSi2O7 True True 4.326700000000001
Solaroxides_June18_206 ZrMnSi2O7 True True 4.3994
Solaroxides_June18_207 ZrMnSi2O7 True True 5.1192
Solaroxides_June18_209 Na2YFeO4 True True 4.2658
Solaroxides_June18_210 Na2YFeO4 True True 4.331200000000001
Solaroxides_June18_216 MnAg(SeO3)2 True True 2.314
Solaroxides_June18_228 Li2TiMnO4 True True 4.1019
Solaroxides_June18_229 Li2TiMnO4 True True 4.0466
Solaroxides_June18_230 Li2TiMnO4 True True 4.1851
Solaroxides_June18_231 Li2TiMnO4 Tru